# Pair Programming 27/05 Regresión lineal múltiple I - Tatiana y Guada

In [1]:
# Importando módulos y cargando el dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings( "ignore", module = "seaborn\..*" )
import sidetable
import statsmodels.api as sm
import pylab as py
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler,OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Ejercicio 1
Toma únicamente las variables de tipo numérico del dataset.
Realiza un EDA explicando algunas de las cosas interesantes del conjunto de datos

Obten la matriz de correlacion de las variables y estudia los resultados explicando las diferentes correlaciones con las variables.

Limpia los datos de trabajo en caso de que sea necesario.

Realiza el ajuste lineal múltiple mostrando las métricas.

# Ejercicio 2
Toma ahora el dataset incluyendo las variables categóricas.
Realiza el EDA de las variables categóricas que se va a incluir en el modelo.

Utiliza algun tipo de codificación para las variables numéricas.

Realiza la misma limpieza de datos, en caso necesario, aplicada en el dataset del apartado para dejar las variables numéricas limpias.

Obten el ajuste lineal múltiple incluyendo las variables categóricas

# Ejercicio 3
Compara las métricas obtenidas del ajuste con las varaibles númerícas y con el modelo que incluia las variables categóricas. Explica los resultados obtenidos, pensando en posibles explicaciones.